### Definitions

In [1]:
from google.colab import drive
drive.mount('/content/drive' , force_remount = True)

Mounted at /content/drive


In [2]:
cd "/content/drive/MyDrive/raw/"

/content/drive/MyDrive/raw


In [3]:
import os
import statistics
import csv
import itertools
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sn
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from keras.models import Sequential
from keras.layers import *
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.utils import plot_model
import seaborn as sns
from matplotlib.colors import rgb2hex
from matplotlib.cm import get_cmap
import plotly
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
from sklearn.preprocessing import StandardScaler, RobustScaler
plotly.offline.init_notebook_mode()

Output hidden; open in https://colab.research.google.com to view.

### MHEALTH

In [4]:
cd "/content/drive/MyDrive/MHEALTHDATASET/"

/content/drive/MyDrive/MHEALTHDATASET


In [5]:
data = pd.read_pickle('mhealth_cnn.pkl')

In [6]:
data = data.rename(columns={'subject': 'subject_id'})
data = data.rename(columns={'activity_id': 'activity'})

In [7]:
from sklearn.model_selection import train_test_split

grouped_data = data.groupby('subject_id')
train_data = pd.DataFrame()
test_data = pd.DataFrame()

for _, group in grouped_data:
    sorted_group = group.sort_index()

    total_samples = len(sorted_group)
    train_size = int(0.8 * total_samples)

    train_group = sorted_group.iloc[:train_size].drop(columns=['subject_id'])
    test_group = sorted_group.iloc[train_size:].drop(columns=['subject_id'])

    train_data = pd.concat([train_data, train_group])
    test_data = pd.concat([test_data, test_group])

data = data.drop(columns=['subject_id'])

### PAMAP2

In [43]:
cd "/content/drive/MyDrive/PAMAP2_Dataset"

/content/drive/MyDrive/PAMAP2_Dataset


In [44]:
data = pd.read_pickle('pamap2.df')

def fix_data(data):
    data = data.drop(data[data['activity_id']==0].index)
    data = data.interpolate()
    for colName in data.columns:
        if data[colName].dtype != 'object':
            data[colName] = data[colName].fillna(data[colName].mean())
    return data

data = fix_data(data)

In [45]:
data = data.drop(columns=['subject_id'])
df_features = data.drop(['activity_id'],axis = 1)#features dataframe
df_target = data['activity_id']

In [ ]:
from sklearn.model_selection import train_test_split

grouped_data = data.groupby('subject_id')
train_data = pd.DataFrame()
test_data = pd.DataFrame()

for _, group in grouped_data:
    sorted_group = group.sort_values(by='time_stamp', ascending=True)

    total_samples = len(sorted_group)
    train_size = int(0.8 * total_samples)

    train_group = sorted_group.iloc[:train_size].drop(columns=['subject_id'])
    test_group = sorted_group.iloc[train_size:].drop(columns=['subject_id'])

    train_data = pd.concat([train_data, train_group])
    test_data = pd.concat([test_data, test_group])

### RF model

In [ ]:
df_features = data.drop(['activity'],axis = 1)#features dataframe
df_target = data['activity']

In [ ]:
#Contaminated
X_train,X_test,y_train,y_test = train_test_split(df_features,df_target,test_size = 0.2,random_state = 42)#test train split
X_train = scaler.fit_transform(X_train)#scaling the train data
X_test = scaler.transform(X_test)

In [ ]:
window_size = 200
stride = 50

train_frames = []
for i in tqdm(range(0, len(train_data)-window_size, stride)):
    window = train_data.iloc[i:i+window_size]
    if window['activity'].nunique() == 1:
      train_frames.append(window)

test_frames = []
for i in tqdm(range(0, len(test_data)-window_size, stride)):
    window = test_data.iloc[i:i+window_size]
    if window['activity'].nunique() == 1:
      test_frames.append(window)

In [ ]:
X_train_list = []
y_train_list = []
relevant_columns = [col for col in train_frames[0].columns if col not in ['subject', 'activity']]

#for each frame replace label with activity
for frame in tqdm(train_frames):
    X_train_list.append(frame[relevant_columns].values)
    y_train_list.append(frame.iloc[0]['activity'])

X_test_list = []
y_test_list = []
for frame in tqdm(test_frames):
    X_test_list.append(frame[relevant_columns].values)
    y_test_list.append(frame.iloc[0]['activity'])

In [19]:
X_train = np.stack(X_train_list)
X_test = np.stack(X_test_list)

y_train = np.array(y_train_list)
y_test = np.array(y_test_list)

In [21]:
num_samples_train, window_size, n_features = X_train.shape
num_samples_test = X_test.shape[0]

In [22]:
X_train_reshaped = X_train.reshape((num_samples_train, window_size * n_features))
X_test_reshaped = X_test.reshape((num_samples_test, window_size * n_features))

In [23]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_reshaped)
X_test_scaled = scaler.transform(X_test_reshaped)

In [28]:
from sklearn.ensemble import RandomForestClassifier
RFmodel = RandomForestClassifier()#creating the model object
RFmodel.fit(X_train_scaled,y_train)

RandomForestClassifier()

### SVM

In [40]:
from sklearn.svm import SVC

SVCmodel = SVC(kernel = 'rbf', probability=True)
SVCmodel.fit(X_train_scaled, y_train)

SVC(probability=True)

### Show Results

In [29]:
def show_Confusion_matrix(y_true,y_predicted):
  cm = confusion_matrix(y_true,y_predicted)
  plt.rcParams['figure.figsize'] = (18,10)#setting the scale to get bigger display
  disp = ConfusionMatrixDisplay(confusion_matrix= cm, display_labels = list_activities)
  disp.plot(xticks_rotation= 'vertical',cmap = 'binary' )
  plt.show()

In [30]:
def get_metrics (y_true,y_pred):# function to get accuracy,precision,Recall and F1 score of the data
    acc = accuracy_score(y_true, y_pred)

    p = precision_score(y_true, y_pred,average=None).mean()#average precision
    r = recall_score(y_true, y_pred, average=None).mean()#average recall
    f1 = f1_score(y_true, y_pred, average=None).mean()#average f1 score

    print("Accuracy:  ",acc)

    print("Precision: ", p)
    print("Recall:    ", r)
    print("F1:        ", f1)

In [46]:
RFmodel_y_pred = SVCmodel.predict(X_test_scaled)#predictions

In [32]:
from sklearn.metrics import precision_score,recall_score, f1_score, confusion_matrix, accuracy_score,ConfusionMatrixDisplay

In [47]:
get_metrics(y_test,RFmodel_y_pred)

Accuracy:   0.4480062548866302
Precision:  0.44060229096494674
Recall:     0.38046520999880007
F1:         0.334390857712845


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.



In [37]:
df_gyr_hand_max = data['ecg_sig_1'].groupby(data['activity']).mean()
list_activities = list(df_gyr_hand_max.index)# just to get the list of 12 activities which are protocol activities

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]